In [69]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from astropy import units as U
from astropy import constants as const
from astropy.coordinates import SkyCoord
import astropy.units as U
import astropy 
from astropy.io import fits
import seaborn as sns
import os

In [59]:
data = pd.read_csv('all_alma_data.csv')


/var/folders/66/76q50s015tq8wc5qz1pvg7y40000gn/T/ipykernel_31753/970427404.py:1: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('all_alma_data.csv')


In [99]:
observation_metadata = data[['s_resolution', 't_exptime', 'em_min', 't_resolution',
                        'em_max', 'em_resolution', 'spatial_resolution', 'bandwidth', 
                        'frequency', 'frequency_support', 'velocity_resolution', 'sensitivity_10kms', 
                        'cont_sensitivity_bandwidth', 'scan_intent', 's_ra', 's_dec', 'antenna_arrays']]
observation_metadata = observation_metadata.dropna()
scan_intents = observation_metadata['scan_intent'].values
ids = []
for i, scan in enumerate(scan_intents):
    if scan == 'TARGET':
        ids.append(i)
ids = np.array(ids)
observation_metadata = observation_metadata.iloc[ids, :]
observation_metadata = observation_metadata[(observation_metadata['s_resolution'] == observation_metadata['spatial_resolution'])]
observation_metadata = observation_metadata.drop(columns=['s_resolution'])     
observation_metadata = observation_metadata[['spatial_resolution', 't_exptime', 'em_min','em_max', 'em_resolution', 
            'bandwidth', 'frequency', 'frequency_support', 'velocity_resolution', 's_ra', 's_dec', 't_resolution', 'antenna_arrays']]

observation_metadata['velocity_resolution'] = observation_metadata['velocity_resolution'] * 10**(-3)
observation_metadata = observation_metadata.rename(columns={'spatial_resolution': 'spatial_resolution [arcsec]',
                                                            't_exptime': 'integration_time [s]',
                                                            'em_resolution': 'frequency_resolution [m]',
                                                            'bandwidth': 'bandwidth [MHz]',
                                                            'frequency': 'frequency [GHz]',
                                                            'velocity_resolution': 'velocity_resolution [Km/s]',
                                                            'em_min': 'frequency_min [m]', 'em_max': 'frequency_max [m]',
                                                            's_ra': 'ra [deg]', 's_dec': 'dec [deg]'})
observation_metadata['bandwidth [MHz]'] = observation_metadata['bandwidth [MHz]'].values * 10**(-6)
coords = SkyCoord(ra=observation_metadata['ra [deg]'], dec=observation_metadata['dec [deg]'], unit='deg').to_string('hmsdms')
freq_sup = observation_metadata['frequency_support'].values
freq_res = [int(float(f.split(',')[1][:-3]))* 10 ** (-3)  for f in freq_sup]
observation_metadata['frequency_resolution [MHz]'] = freq_res
b = 'J2000 '
coords = [b + i for i in coords]
observation_metadata['coords'] = coords
antenna_arrays = observation_metadata['antenna_arrays'].values
antenna_arrays = [[a.split(':')[0] for a in antenna_arrays[i].split(' ')] for i in range(len(antenna_arrays))]
print(antenna_arrays[0])
print(type(antenna_arrays[0]))
observation_metadata['pads'] = antenna_arrays
to_save = observation_metadata[['spatial_resolution [arcsec]', 'integration_time [s]', 
                                        'velocity_resolution [Km/s]', 'ra [deg]', 'dec [deg]', 'bandwidth [MHz]', 'frequency [GHz]', 'pads']]
to_save.to_csv('obs_configurations.csv', index=False)

['J501', 'J502', 'J503', 'J504', 'J506', 'N601', 'N602', 'N603', 'N605']
<class 'list'>
<class 'list'>


In [109]:
def create_antennas_dict():
    antenna_file = open('/Users/michele/Documents/GitHub/ALMASim/alma.all.cfg', 'r')
    antenna_dict = {}
    for line in antenna_file:
        if '#' not in line:
            x, y, z, dim, pad = line.split()
            antenna_dict[pad] = [x, y, z, dim]
    antenna_file.close()
    return antenna_dict

antenna_dict = create_antennas_dict()

obs_db = pd.read_csv('obs_configurations.csv')
n = 10
print(type(obs_db['pads'].values[0]))
params = obs_db.sample(n=n, axis=0)
antenna_configs = params['pads'].values
spatial_res = params['spatial_resolution [arcsec]'].values
test_config = antenna_configs[0]
print(test_config)
print(type(test_config))


<class 'str'>
['A001', 'A003', 'A004', 'A006', 'A007', 'A008', 'A011', 'A015', 'A017', 'A020', 'A021', 'A029', 'A037', 'A038', 'A046', 'A050', 'A053', 'A067', 'A068', 'A069', 'A070', 'A071', 'A072', 'A075', 'A077', 'A082', 'A137', 'A138', 'T701']
<class 'str'>


In [110]:
test_config = test_config[1:-1].split(', ')
test_config = [a[1:-1] for a in test_config]
print(test_config)

['A001', 'A003', 'A004', 'A006', 'A007', 'A008', 'A011', 'A015', 'A017', 'A020', 'A021', 'A029', 'A037', 'A038', 'A046', 'A050', 'A053', 'A067', 'A068', 'A069', 'A070', 'A071', 'A072', 'A075', 'A077', 'A082', 'A137', 'A138', 'T701']


In [111]:
beginning_lines = ['# observatory=ALMA', '# coordsys=LOC (local tangent plane)', '# x y z diam pad#']
i = 0
with open(os.path.join('/Users/michele/Documents/GitHub/ALMASim/antenna_config', 'antenna_config_{}.cfg'.format(str(i))), 'w') as f:
    for line in beginning_lines:
        f.write(line)
        f.write('\n')
    for antenna in test_config:
        print(antenna)
        x, y, z, dim = antenna_dict[antenna]
        print(x, y, z, dim)
        f.write('{} {} {} {} {}'.format(x, y, z, dim, antenna))
        f.write('\n')
    f.close()

A001
5.606192499 7.646657746 -2.087775605 12.
A003
16.9413543 28.41476916 -2.082643323 12.
A004
34.07993959 -3.391862608 -2.089849501 12.
A006
60.44668677 -1.065310291 -2.089494285 12.
A007
55.4353013 19.72317522 -2.087556286 12.
A008
48.98284 33.32294429 -2.489031714 12.
A011
77.32651615 -15.4773776 -2.239047365 12.
A015
50.24060942 60.83113893 -2.48917588 12.
A017
32.78896049 35.44741771 -2.087282025 12.
A020
-21.55636298 -2.392685398 -1.689675267 12.
A021
-4.25931373 28.22428908 -2.088522333 12.
A029
-31.50495352 64.57049807 -1.887558625 12.
A037
-11.83400494 -26.27567729 -2.084995641 12.
A038
-40.99650541 -44.71276927 -1.693272277 12.
A046
2.833032901 -41.76763393 -2.090960204 12.
A050
24.45367016 -43.93559429 -2.086322948 12.
A053
-5.974014263 -101.9648369 -1.586918413 12.
A067
123.8238479 22.29677914 -3.742713407 12.
A068
-101.0065464 69.23582577 -0.398483196 12.
A069
-120.0516036 -69.06984556 0.167789404 12.
A070
147.5980027 -42.46823003 -3.846486286 12.
A071
-79.3666209 137.789